# Machine Learning

In this file, instructions how to approach the challenge can be found.

We are going to work on different types of Machine Learning problems:

- **Regression Problem**: The goal is to predict delay of flights.
- **(Stretch) Multiclass Classification**: If the plane was delayed, we will predict what type of delay it is (will be).
- **(Stretch) Binary Classification**: The goal is to predict if the flight will be cancelled.

In [141]:
import pandas as pd
from scipy.stats import shapiro
from scipy.stats import ttest_1samp
import numpy as np
import psycopg2
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_squared_error 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

In [142]:
test = pd.read_csv('flights_test.csv')

In [143]:
df = pd.read_csv('flights_merged.csv', index_col=0)

In [144]:
df = df.dropna(axis=0, subset=['crs_elapsed_time'])

In [159]:
df = df.sample(frac=0.1)

In [160]:
df

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,distance,passengers,capacity,total_gallons,dep_delay,nas_delay,security_delay,late_aircraft_delay,weather_delay,carrier_delay
619085,2019-07-22,WN,WN,WN,2206,WN,N250WN,2206,14635,RSW,...,919.0,2167.666667,924.000000,1.723195e+08,58.890244,8.189024,0.000000,35.274390,1.609756,9.798780
320120,2019-01-13,AA,AA,AA,872,AA,N93003,872,12266,IAH,...,964.0,119.000000,62.000000,3.079852e+08,43.928571,11.721805,0.097744,24.165414,1.639098,12.454887
381846,2019-09-22,AS,AS,AS,828,AS,N508AS,828,10299,ANC,...,2777.0,3745.000000,1025.000000,6.101888e+07,37.761905,10.333333,0.000000,14.285714,0.000000,18.904762
788776,2019-07-03,F9,F9,F9,502,F9,N318FR,502,11292,DEN,...,1620.0,1922.250000,203.250000,2.353042e+07,39.877193,36.263158,0.000000,9.666667,1.631579,2.807018
624999,2019-07-08,WN,WN,WN,768,WN,N7717D,768,14771,SFO,...,447.0,11223.000000,3482.500000,1.723195e+08,60.090909,5.023923,0.002392,41.277512,0.610048,9.131579
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324460,2019-02-20,AA,AA,AA,2423,AA,N390AA,2423,11298,DFW,...,1391.0,572.500000,118.500000,3.079852e+08,61.485106,19.514894,0.000000,20.055319,3.463830,21.357447
225241,2019-06-17,AA,AA_CODESHARE,AA,4155,MQ,N264NN,4155,11278,DCA,...,562.0,130.000000,49.000000,0.000000e+00,56.665072,10.100478,0.000000,34.009569,2.430622,15.507177
337055,2019-12-09,AA,AA_CODESHARE,AA,5695,OH,N556NN,5695,12339,IND,...,588.0,80.000000,80.000000,0.000000e+00,45.072727,23.466667,0.036364,19.696970,1.872727,7.151515
331864,2019-01-15,AA,AA_CODESHARE,AA,4408,YX,N434YX,4408,15016,STL,...,1068.0,4176.000000,816.000000,2.209100e+04,38.000000,16.056338,0.000000,10.619718,1.084507,20.492958


In [161]:
y= df['arr_delay']

In [162]:
X = df

In [163]:
X = X.drop(columns=['arr_delay'])

In [88]:
#X.fl_date = X.fl_date.apply(pd.to_datetime)

## Main Task: Regression Problem

The target variable is **ARR_DELAY**. We need to be careful which columns to use and which don't. For example, DEP_DELAY is going to be the perfect predictor, but we can't use it because in real-life scenario, we want to predict the delay before the flight takes of --> We can use average delay from earlier days but not the one from the actual flight we predict.  

For example, variables **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY** shouldn't be used directly as predictors as well. However, we can create various transformations from earlier values.

We will be evaluating your models by predicting the ARR_DELAY for all flights **1 week in advance**.

### Feature Engineering

Feature engineering will play a crucial role in this problems. We have only very little attributes so we need to create some features that will have some predictive power.

- weather: we can use some weather API to look for the weather in time of the scheduled departure and scheduled arrival.
- statistics (avg, mean, median, std, min, max...): we can take a look at previous delays and compute descriptive statistics
- airports encoding: we need to think about what to do with the airports and other categorical variables
- time of the day: the delay probably depends on the airport traffic which varies during the day.
- airport traffic
- unsupervised learning as feature engineering?
- **what are the additional options?**: Think about what we could do more to improve the model.

In [164]:
X = X[['origin_airport_id', 'dest_airport_id', 'air_time', 'distance', 'passengers', 'crs_elapsed_time','capacity', 'total_gallons', 'dep_delay',
       'nas_delay', 'security_delay', 'late_aircraft_delay',
       'weather_delay', 'carrier_delay']]
#dropped flight date :( 

In [165]:
scaler = StandardScaler()

In [166]:
X = pd.DataFrame(scaler.fit_transform(X))

In [167]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=0.8)


### Feature Selection / Dimensionality Reduction

We need to apply different selection techniques to find out which one will be the best for our problems.

- Original Features vs. PCA conponents?

### Modeling

Use different ML techniques to predict each problem.

- linear / logistic / multinomial logistic regression
- Naive Bayes
- Random Forest
- SVM
- XGBoost
- The ensemble of your own choice

In [92]:
#data_dmatrix = xgb.DMatrix(data=X,label=y, enable_categorical=True)

In [93]:
#objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1, max_depth = 7, alpha = 0.1,

In [119]:
lr = LinearRegression()

In [120]:
lr.fit(X_train, y_train) 

LinearRegression()

In [121]:
lr.score(X_train, y_train)

0.03795838486980141

In [122]:
lr.score(X_test, y_test)

0.03657385654892786

### Random Forest

RandomForestRegressor()

In [168]:
 # create regressor object
regressor = RandomForestRegressor(n_estimators = 100)
 

In [169]:
# fit the regressor with x and y data
regressor.fit(X_train, y_train) 

RandomForestRegressor()

In [170]:
y_pred = regressor.predict(X_test)

In [171]:
print('Mean Absolute Error:', mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 33.3062775799608
Mean Squared Error: 1957.192259450817
Root Mean Squared Error: 44.24016568064384


In [172]:
params = {'n_estimators' : [100, 150, 200, 300], 'max_depth': [3, 10, 20, 30]}

In [173]:
grid = GridSearchCV(regressor, params)

In [ ]:
grid_result = grid.fit(X_train, y_train)

In [ ]:
print(grid_result.best_score_)
print( grid_result.best_params_['n_estimators'])
print( grid_result.best_params_['max_depth'])

### Evaluation

You have data from 2018 and 2019 to develop models. Use different evaluation metrics for each problem and compare the performance of different models.

You are required to predict delays on **out of sample** data from **first 7 days (1st-7th) of January 2020** and to share the file with LighthouseLabs. Sample submission can be found in the file **_sample_submission.csv_**

======================================================================
## Stretch Tasks

### Multiclass Classification

The target variables are **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY**. We need to do additional transformations because these variables are not binary but continuos. For each flight that was delayed, we need to have one of these variables as 1 and others 0.

It can happen that we have two types of delays with more than 0 minutes. In this case, take the bigger one as 1 and others as 0.

### Binary Classification

The target variable is **CANCELLED**. The main problem here is going to be huge class imbalance. We have only very little cancelled flights with comparison to all flights. It is important to do the right sampling before training and to choose correct evaluation metrics.